# Sentinel-1 EW RTC with pyroSAR (SNAP)

**Overview**
- Notebook for creating Radiometrically Terrain Corrected (RTC) data for Sentinel-1 EW using the pyroSAR software
- This notebook will search and download the following data:
    - Sentinel-1 SLC 
    - Precise Orbit Files
    - Copernicus 30m DEM covering region of Interest
- RTC backscatter will be created with the pyroSAR software package (SNAP backend)

**Requirements**
- A conda environment setup as described in the *README.md*
- Appropriate storage space
- Credentials for usgs (https://urs.earthdata.nasa.gov/users/new) 
- Credentials for copernicus CDSE ( https://dataspace.copernicus.eu/)



# Imports

In [ ]:
# initial setup
import os
import asf_search as asf
from eof.download import download_eofs
import yaml
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import shutil
from shapely import Polygon, box
import numpy as np
import pyproj
import rasterio
import rasterio.merge
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyroSAR.snap import geocode

# Settings

In [ ]:
print(os.environ['PATH'])

### Set the path the the snap binary

In [ ]:
# Set the path the the snap binary
# Default location on NCI is $HOME/snap/bin
# Can also export from command line :
# export PATH="$PATH:$HOME/snap/bin"
SNAP_PATH = '/home/547/ab7271/snap/bin'
if 'snap' not in os.environ['PATH']:
    os.environ['PATH'] = os.environ['PATH'] + ':' + SNAP_PATH
print(os.environ['PATH'])

### Set directory paths

In [ ]:
# paths
data_dir =  '/g/data/yp75/projects/sar-antractica-processing/alexf_ew/data' #'/home/547/ab7271/s1-rtc-isce3-notebook/data' # set this as the absolute path
download_folder = os.path.join(data_dir,'scenes')
earthdata_credentials_path = "credentials/credentials_earthdata.yaml"
copernicus_credentials_path = "credentials/credentials_copernicus.yaml"
precise_orb_download_folder = os.path.join(data_dir,'orbits','POEORB')
restituted_orb_download_folder = os.path.join(data_dir,'orbits','RESORB')
rtc_outpath = os.path.join(data_dir,'processed_scene_snap')
rtc_scratch_dir = os.path.join(data_dir,'temp')
dem_download_folder = os.path.join(data_dir,'dem')

In [ ]:
create_folders = True
if create_folders:
    for f in [
        download_folder, 
        precise_orb_download_folder, 
        restituted_orb_download_folder,
        dem_download_folder,
        rtc_outpath,
        rtc_scratch_dir
        ]:
        os.makedirs(f, exist_ok=True)

# Credentials

In [ ]:
with open(earthdata_credentials_path, "r", encoding='utf8') as f:
        earthdata_credentials = yaml.safe_load(f.read())
with open(copernicus_credentials_path, "r", encoding='utf8') as f:
        copernicus_credentials = yaml.safe_load(f.read())

# Search and Download Scene of Interest

### Option 1 - Search based on a geometry and daterange

In [ ]:
wkt = "POINT (130.7085216 -16.0044816)" # South Australia
wkt = "POINT (110.526792 -66.282343)" # Casey station Antarctica
print(f'Searching over point: {wkt}')

In [ ]:
# product type https://github.com/asfadmin/Discovery-asf_search/blob/master/asf_search/constants/PRODUCT_TYPE.py
# prod = 'GRD_HS' # IW
prod = 'GRD_MD' # EW

# prod = 'GRD_HD'
# prod = 'GRD_MS'
# prod = 'GRD_FD'
# prod = 'SLC'

# acquisition mode
#mode = 'IW'
mode = 'EW'

results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                     intersectsWith=wkt, 
                     maxResults=1, 
                     processingLevel=prod,
                     beamMode=mode,
                     start='2021-12-31T11:59:59Z',
                     end='2022-01-19T11:59:59Z')
print(len(results))

### Option 2 - Search for known products

In [ ]:
search_list = True
if search_list:

    # prod = 'SLC'
    # mode = 'IW'
    prod = 'GRD_MD'
    mode = 'EW'
    
    scene_list = [
        #'S1A_EW_GRDM_1SDH_20210701T150428_20210701T150533_038585_048D91_23E6', 
        'S1A_EW_GRDM_1SDH_20210704T152833_20210704T152942_038629_048EE8_D41F'
   ]

    # search the scene list with the specified product type
    results = asf.granule_search(scene_list, asf.ASFSearchOptions(processingLevel=prod, beamMode=mode))

In [ ]:
for i, r in enumerate(results):
    print(f"{i+1} - {r.properties['sceneName']}")
    print(r.properties)
    print(r.geometry)
    print('\n')

## Download Scenes

In [ ]:
# download
session = asf.ASFSession()
session.auth_with_creds(earthdata_credentials['login'],earthdata_credentials['password'])

In [ ]:
# limit to the first scene
results = results[0:1]
geometry = results[0].geometry
scene = results[0].properties['sceneName']
pol = results[0].properties['polarization']
name = str(results[0].properties['sceneName'])

In [ ]:
print(f'Downloading {len(results)} scenes')

In [ ]:
# download all results
scene_paths = []
scene_names = []
for s in results:
    name = s.properties['sceneName']
    scene_names.append(name)
    print(name)
    path = os.path.join(download_folder, name)
    s.download(path=download_folder, session=session)
    scene_paths.append(path)

## Download Precise Orbit Files

In [ ]:
download_orbits = True
# download not required, happens in pyrosar geocode process
if download_orbits:
    for scene in  scene_paths:
        # download precise orbits
        download_eofs(
            sentinel_file=scene, 
            save_dir=precise_orb_download_folder, 
            orbit_type='precise', 
            cdse_user=copernicus_credentials['login'], 
            cdse_password=copernicus_credentials['password'],
            asf_user=earthdata_credentials['login'], 
            asf_password=earthdata_credentials['password']
            )
        # download restituted orbits
        download_eofs(
            sentinel_file=scene, 
            save_dir=restituted_orb_download_folder, 
            orbit_type='restituted', 
            cdse_user=copernicus_credentials['login'], 
            cdse_password=copernicus_credentials['password'],
            asf_user=earthdata_credentials['login'], 
            asf_password=earthdata_credentials['password']
            )

In [ ]:
print('precise orbits')
for p in os.listdir(precise_orb_download_folder):
    prec_orb_path = os.path.join(precise_orb_download_folder,p)
    print(prec_orb_path)

print('restited orbits')
for p in os.listdir(restituted_orb_download_folder):
    res_orb_path = os.path.join(restituted_orb_download_folder,p)
    print(res_orb_path)

# Download and process DEMs

## Helper functions

In [ ]:
def find_required_dem_tile_paths_by_filename(
    bounds: tuple,
    check_exists: bool = True,
    cop30_folder_path = '/g/data/v10/eoancillarydata-2/elevation/copernicus_30m_world/',
    search_buffer=0.3,
    tifs_in_subfolder=True,
) -> list[str]:
    """generate a list of the required dem paths based on the bounding coords. The
    function searches the specified folder.

    Parameters
    ----------
    bounds : tuple
        the set of bounds (min_lon, min_lat, max_lon, max_lat)
    check_exists : bool, optional
        Check if the file exists, by default True
    cop30_folder_path : str, optional
        path to the tile folders, by default COP30_FOLDER_PATH

    Returns
    -------
    list[str]
        list of paths for required dem tiles in bounds
    """

    # add a buffer to the search
    bounds = box(*bounds).buffer(search_buffer).bounds
    # logic to find the correct files based on data being stored in each tile folder
    min_lat = np.floor(bounds[1]) if bounds[1] < 0 else np.ceil(bounds[1])
    max_lat = np.ceil(bounds[3]) if bounds[3] < 0 else np.floor(bounds[3]) + 1
    min_lon = np.floor(bounds[0]) if bounds[0] < 0 else np.floor(bounds[0])
    max_lon = np.ceil(bounds[2]) if bounds[2] < 0 else np.ceil(bounds[2])
    lat_range = list(range(int(min_lat), int(max_lat)))
    lon_range = list(range(int(min_lon), int(max_lon)))
    print(f"lat tile range: {lat_range}")
    print(f"lon tile range: {lon_range}")
    dem_paths = []
    dem_folders = []

    for lat in lat_range:
        for lon in lon_range:
            lat_dir = "N" if lat >= 0 else "S"
            lon_dir = "E" if lon >= 0 else "W"
            dem_foldername = f"Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{lon_dir}{int(abs(lon)):03d}_00_DEM"
            if tifs_in_subfolder:
                dem_subpath = f"{dem_foldername}/{dem_foldername}.tif"
            else:
                dem_subpath = f"{dem_foldername}.tif"
            dem_path = os.path.join(cop30_folder_path, dem_subpath)
            if check_exists:
                # check the file exists, e.g. over water will not be a file
                if os.path.exists(dem_path):
                    dem_paths.append(dem_path)
                    dem_folders.append(dem_foldername)
            else:
                dem_paths.append(dem_path)
    return sorted(list(set(dem_paths)))

In [ ]:
# buffer the sceme nounda to ensure coverage
scene_bounds = Polygon(geometry['coordinates'][0]).bounds
print(f'original scene bounds : {scene_bounds}')

## Adjust DEM bounds to ensure area is covered
- Note at high latitudes this may not be sufficient

In [ ]:
buffer_degrees = 1
scene_poly = box(*scene_bounds)
expanded_scene_boounds = scene_poly.buffer(buffer_degrees).bounds
# Create the polygon
geom = Polygon([(expanded_scene_boounds[0], expanded_scene_boounds[1]), 
                (expanded_scene_boounds[2], expanded_scene_boounds[1]), 
                (expanded_scene_boounds[2], expanded_scene_boounds[3]), 
                (expanded_scene_boounds[0], expanded_scene_boounds[3])])
expanded_scene_geom = list(geom.exterior.coords)

In [ ]:
print(f'original bounds : {scene_bounds}')
print(f'expanded bounds : {expanded_scene_boounds}')


In [ ]:
min_lat, max_lat  = min([c[1] for c in expanded_scene_geom]), max([c[1] for c in expanded_scene_geom])
min_lon, max_lon  = min([c[0] for c in expanded_scene_geom]), max([c[0] for c in expanded_scene_geom])
lats = list(range(int(np.floor(min_lat)), int(np.ceil(max_lat+1))))
longs = list(range(int(np.floor(min_lon)), int(np.ceil(max_lon+1))))
print(expanded_scene_geom)
print(lats)
print(longs)

## Option 1 - Find and download Copernicus 30m DEM from AWS

In [ ]:
dem_s3_paths = []

for lat in lats:
    for long in longs:
        if lat >= 0:
            lat_dir = "N"
        else:
            lat_dir = "S"
        if long >= 0:
            long_dir = "E"
        else:
            long_dir = "W"
        
        dem_s3_paths.append(f"Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM/Copernicus_DSM_COG_10_{lat_dir}{int(abs(lat)):02d}_00_{long_dir}{int(abs(long)):03d}_00_DEM.tif")
dem_s3_paths = list(set(dem_s3_paths))
print(f'DEM tiles to download : {len(dem_s3_paths)}')
#dem_s3_paths

In [ ]:
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "copernicus-dem-30m"
bucket = s3.Bucket(bucket_name)
dems_to_merge = []

for s3_path in dem_s3_paths:
    try:
        dl_path = os.path.join(dem_download_folder,s3_path.split('/')[1])
        if not os.path.exists(dl_path):
            bucket.download_file(s3_path, dl_path)
            print(f'downloaded : {s3_path}')
            dems_to_merge.append(dl_path)
        else:
            print(f'file exists : {s3_path}')
            dems_to_merge.append(dl_path)
    except:
        print(f'not found : {s3_path}')

In [ ]:
dem_paths = ' '.join(dems_to_merge)
dem_paths

# Option 2 - Use Local Cop30m Data

In [ ]:
COP30_FOLDER_PATH = "/g/data/v10/eoancillarydata-2/elevation/copernicus_30m_world/"

In [ ]:
print(expanded_scene_boounds)
dem_paths = find_required_dem_tile_paths_by_filename(expanded_scene_boounds, search_buffer=0,cop30_folder_path=COP30_FOLDER_PATH)
print(f'Number of DEMS to merge {len(dem_paths)}')
dem_paths = ' '.join(dem_paths)

In [ ]:
dem_paths

## Merge the DEM

In [ ]:
merged_dem_path = os.path.join(dem_download_folder,f"{name}_dem.tif")

In [ ]:
# set the proj location if it raises a warning
os.environ['PROJ_LIB'] = '/g/data/yp75/ab7271/microconda/envs/pyrosar_rtc/share/proj'
! echo $PROJ_LIB
!gdal_merge.py -n 0 -o $merged_dem_path $dem_paths

In [ ]:
def fix_no_data_value(input_file, output_file, no_data_value=-9999):
    with rasterio.open(input_file, "r+") as src:
        nodata = src.nodata
        if nodata in [None, np.nan, 'nan','np.nan','np.Nan']:
            print(f'replace dem nodata from np.nan to -9999')
            src.nodata = no_data_value
            with rasterio.open(output_file, 'w',  **src.profile) as dst:
                dem_data = dst.read(1)
                dem_data[dem_data==np.nan] = -9999
                dem_data[dem_data=='nan'] = -9999
                dem_data[dem_data==None] = -9999
                dst.write(dem_data, 1)
                dst.update_tags(AREA_OR_POINT='Point')

In [ ]:
# pyroSAR cant handle a nodata value of np.nan
# we therefore set this to be -9999
fix_no_data_value(merged_dem_path, merged_dem_path, no_data_value=-9999)

# Produce PyroSAR RTC

In [ ]:
# settings
spacing = 40
scaling = 'linear' # scale of final product, linear, db
refarea = 'gamma0' # e.g. gamma0, sigma0, beta0 or ['gamma0','sigma0']
t_crs = 3031
export_extra = ["localIncidenceAngle","DEM","layoverShadowMask","scatteringArea","gammaSigmaRatio"]

In [ ]:
# provide args to snap/java to limit memory etc (provide as list)
# https://forum.step.esa.int/t/gpt-and-snap-performance-parameters-exhaustive-manual-needed/8797 
# https://github.com/johntruckenbrodt/pyroSAR/blob/main/pyroSAR/snap/util.py#L158
gpt_args = ['-x']
# gpt_args = []

In [ ]:
# note, you may need to clear the results folder if the scene has been run before
# note function downloads orbits, might cause issue with batch NCI run
scene_workflow = geocode(
    infile=os.path.join(download_folder,f"{scene}.zip"),
    outdir=rtc_outpath,
    allow_RES_OSV=True,
    externalDEMFile=merged_dem_path,
    externalDEMNoDataValue=-9999,
    externalDEMApplyEGM=True, 
    spacing=spacing,
    scaling=scaling,
    refarea=refarea,
    t_srs=t_crs,
    returnWF=True,
    clean_edges=True,
    export_extra=export_extra,
    gpt_args=gpt_args,
            )

Change some parameters of the config to point to our files

- The run is based off of this .yaml file, so there may be additonal changes you want to make

- For example, additional metadata, projections and output resolutions